In [3]:
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.optim as optim
from torchvision import transforms,models
import time

In [9]:

vggList = [models.vgg11(pretrained=True).features,models.vgg13(pretrained=True).features,models.vgg16(pretrained=True).features,models.vgg19(pretrained=True).features]

for vgg in vggList:
    
    for param in vgg.parameters():
        param.requires_grad_(False)
    
    devices=torch.device("cuda" if torch.cuda.is_available() else "cpu")

    vgg.to(devices)
    print(devices)


cuda
cuda
cuda
cuda


In [ ]:
def style_loss(Y_hat, gram_Y):
    return np.square(gram(Y_hat) - gram_Y).mean()

def content_loss(Y_hat, Y):
    return np.square(Y_hat - Y).mean()

def gram(X):
    num_channels, n = X.shape[1], X.size // X.shape[1]
    X = X.reshape(num_channels, n)
    return np.dot(X, X.T) / (num_channels * n)

def compute_loss(X, contents_Y_hat, styles_Y_hat, contents_Y, styles_Y_gram):
    # Calculate the content, style, and total variance losses respectively
    contents_l = [content_loss(Y_hat, Y) * content_weight for Y_hat, Y in zip(
        contents_Y_hat, contents_Y)]
    styles_l = [style_loss(Y_hat, Y) * style_weight for Y_hat, Y in zip(
        styles_Y_hat, styles_Y_gram)]
    tv_l = tv_loss(X) * tv_weight
    # Add up all the losses
    l = sum(styles_l + contents_l + [tv_l])
    return contents_l, styles_l, tv_l, l

In [ ]:
def a():
    
    start = time.time()
    show_every=400

    optimizer=optim.Adam([target],lr=0.003)
    steps=2000

    for ii in range(1,steps+1):
        target_features=get_features(target,vgg)
        content_loss=torch.mean((target_features['conv2_2']-content_features['conv2_2'])**2)

        style_loss=0
        for layer in style_weights:
            target_feature=target_features[layer]
            #print(target_feature)
            target_gram=gram_matrix(target_feature)
            _,d,h,w=target_feature.shape
            style_gram=style_grams[layer]
            layer_style_loss=style_weights[layer]*torch.mean((target_gram-style_gram)**2)
            style_loss+=layer_style_loss/(d*h*w)

        total_loss=content_weight*content_loss+style_weight*style_loss

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        if ii% show_every==0:
            print('Total loss:',total_loss.item())
            plt.imshow(im_convert(target))
            plt.show()
            print('Time:',time.time() - start)
    print(time.time() - start)

# vgg11

# vgg13

# vgg16

# vgg19